# TIME SERIES FORCASTING 

1. [Project Scope](#1)
2. [What is Time series?](#1)
3. [Components of Time series forecasting](#2)
4. [Let’s step back  : Basics of modeling a time series data](#3)
5. [How to approach a forecasting problem](#4)   
6. [Foundation of popular algorithms like ARIMA / SARIMA](#5)
7. [Getting the data](#6)
8. [Forecasting using SARIMAX and LSTM ](#7)
9. [Evaluation](#8)

## Project Scope

1. **Objective** : To segment customers as per their shopping patterns and predict future footfalls.
2. **What** : Forecast number of trips at different levels across stores/online identify patterns
3. **Why**: To estimate futuristic trips and drive traffic

    
## What is time series 

Time Series Modeling. As the name suggests, it involves working on time (years, days, hours, minutes) based data, to derive hidden insights to make informed decision making.

<a id="2"></a>

## Components of Time series forecasting

1. **Trend**: Series could be constantly increasing or decreasing or first decreasing for a considerable period and then decreasing. 
    his trend is identified and then removed from the time series in ARIMA forecasting process.

2. **Seasonality** : Repeating pattern with fixed period.
Example - Sales in festive seasons. Sales of Candies and sales of Chocolates peaks in every October Month and 
December month respectively every year in US. It is because of Halloween and Christmas falling in those months. 
The time-series should be de-seasonalized in ARIMA forecasting process.

3. **Random Variation** (Irregular Component)
This is the unexplained variation in the time-series which is totally random.
Erratic movements that are not predictable because they do not follow a pattern. Example – Earthquake

<img src = "pic4.jpg" align = "left" height = "300" width="300">

In [7]:
from statsmodels.tsa.seasonal import seasonal_decompose
import pandas as pd
df = pd.read_csv('master_data.csv')
df.head()
y = df[['trips']]
feature = ['Holiday_Flag', 'Promotion_Flag']
result = seasonal_decompose(y, model='additive',freq = 1)

<a id="3"></a>

## Basics – Time Series Modeling

a. **Stationary Series**: stationary series is one whose mean and variance of the series is constant over time. 
To calculate the expected value, we generally take a mean across time intervals. 
The mean across many time intervals makes sense only when the expected value is the same across those time periods. 
If the mean and population variance can vary, there is no point estimating by taking an average across time. 
In cases where the stationary criterion are violated, the first requisite becomes to stationaries’ 
the time series and then try stochastic models to predict this time series.
There are multiple ways of bringing this stationarity. Some of them are Detrending, Differencing etc.
 

<img src = "stationary.jpg" align = "left" width="400">

In [ ]:
df['trips'].plot(x='index')

b. **Stationarity Test** : Augmented Dickey Fuller Test

In [ ]:
### STATIONARITY TEST

series = df.loc[:, 'trips'].values
result = adfuller(series, autolag='AIC')
print(f'ADF Statistic: {result[0]}')
print(f'n_lags: {result[1]}')
print(f'p-value: {result[1]}')
for key, value in result[4].items():
    print('Critial Values:')
    print(f'   {key}, {value}')    
    
#The p-value is obtained is greater than significance level of 0.05 and the ADF statistic is higher than any of
#the critical values.

c. **Eliminate Trend and seasonality**
1. Differencing – taking the difference with a particular time lag
2. Decomposition – modeling both trend and seasonality and removing them from the model.

<a id="4"></a>

## How to approach Time series problem:
1. Naïve Approach        
2. Simple Average 
3. Moving average 
4. Simple Exponential Smoothing 
5. Holt’s Linear Trend Method  
6. Holt Filter
7. ARIMA

<img src = "pic1.jpg" align = "left" height = "600" width="600">

<img src = "pic2.jpg" align = "left" height= "400" width="400">

<a id="5"></a>

**ARIMA in detail**

**Auto-regressive component**
It implies relationship of a value of a series at a point of time with its own previous values. Such relationship can exist with any order of lag.
Lag - Lag is basically value at a previous point of time. It can have various orders as shown in the table below. It hints toward a pointed relationship.

**Moving average components**
It implies the current deviation from mean depends on previous deviations. Such relationship can exist with any number of lags which decides the order of moving average.
Moving Average - Moving Average is average of consecutive values at various time periods.  It can have various orders as shown in the table below. It hints toward a distributed relationship as moving itself is derivative of various lags.

                        
**Difference between AR and MA models**
The primary difference between an AR and MA model is based on the correlation between time series objects at different time points. The correlation between x(t) and x(t-n) for n > order of MA is always zero. This directly flows from the fact that covariance between x(t) and x(t-n) is zero for MA models (something which we refer from the example taken in the previous section). However, the correlation of x(t) and x(t-n) gradually declines with n becoming larger in the AR model. This difference gets exploited irrespective of having the AR model or MA model. The correlation plot can give us the order of MA model.


**What order of AR or MA process do we need to use?**

The first question can be answered using Total Correlation Chart (also known as Auto – correlation Function / ACF). ACF is a plot of total correlation between different lag functions. For instance, in GDP problem, the GDP at time point t is x(t). We are interested in the correlation of x(t) with x(t-1) , x(t-2) and so on. Now let’s reflect on what we have learnt above.
In a moving average series of lag n, we will not get any correlation between x(t) and x(t – n -1) . Hence, the total correlation chart cuts off at nth lag. So, it becomes simple to find the lag for a MA series. For an AR series, this correlation will gradually go down without any cut off value. So what do we do if it is an AR series?
Here is the second trick. If we find out the partial correlation of each lag, it will cut off after the degree of AR series. For instance, if we have a AR(1) series,  if we exclude the effect of 1st lag (x (t-1) ), our 2nd lag (x (t-2) ) is independent of x(t). Hence, the partial correlation function (PACF) will drop sharply after the 1st lag. 

**Introducing SARIMAX**
When the ARIMA model is further applied to the seasonal part, we get Seasonal ARIMA. ARIMA generates p,d,q values for 
stationary data, SARIMAX produces p,d,q  values for the seasonal part separately as well.

<a id="6"></a>

<a id="7"></a>

## Forecasting code using Sarimax and Lstm

#### SARIMAX

In [ ]:
from pmdarima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX 
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

    
stepwise_model = auto_arima(df['trips'],exogenous= X[['Holiday_Flag', 'Promotion_Flag']]
                            ,start_p=0, start_q=0,
                         max_p=5, max_q=5, m=1,
                        start_P=0, seasonal=True,
                           d=0, D=1, trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=True)
stepwise_model.summary()  

#Train Test Split
train = df[0:776]
X_train = train[feature]
y_train = train[[target]]

test = df[776:779]
X_test = test[feature]
y_test = test[[target]]


# Get the order values from stepwise_model.summary()
model = SARIMAX(y_train[[target]], exogenous = X_train[feature],
                order = (4, 0, 1),  
                seasonal_order =(0, 0, 0, 1)) 
  
result = model.fit() 


start = len(train) 
end = len(train) + len(test) - 1 

predictions  = result.predict(start,end,return_conf_int=False).rename("Predictions") 


#Performance measurement
output = pd.concat([test, predictions], axis=1)
output.head()
output['mape'] = abs(output['Predictions']-output['dmd_dol'])/output['dmd_dol']
mape = output['mape']
mape.mean() * 100



#### LSTM  using Keras

In [ ]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense,LSTM
import matplotlib.pyplot as plt
from keras.layers.core import Dense, Activation, Dropout
seed = 7
np.random.seed(seed)


def to_supervised(df,dropNa = True,lag=1):
    #df = pd.DataFrame(data)
        column = []
        column.append(df)
        for i in range(1,lag+1):
            column.append(df.shift(-i))  #lags added here
        df = pd.concat(column,axis=1)
        df.dropna(inplace = True)
        features = merged.shape[1]
        df = df.values
        supervised_data = df[:,:features*lag]
        supervised_data = np.column_stack( [supervised_data, df[:,features*lag]])
        return supervised_data

    supervised = to_supervised(merged,lag=timeSteps)
    dataframe = pd.DataFrame(supervised)
    
    
    df = dataframe.rename({0: 'trips',  1:'Promotion_Flag',2:'Holiday_flag' 2:'lag_trips'}, axis=1)
    ## Notice the lag trips, try to visualize the dataframe first.


    cols_to_scale = ['trips','lag_trips']

    scaler = MinMaxScaler(feature_range=(0,1))
    scaled_cols = scaler.fit_transform(df[cols_to_scale])
    df[cols_to_scale] = scaled_cols

    X = df.drop(['lag_trips'], axis=1)
    Y = df['lag_trips']
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.205, shuffle=False)
    #return X_test

#Rest col : check if rest columns calculated correctly

    rest_col=rest_col[['cm','gross_amt']]
    dummy=rest_col['cm']
    rest_train, rest_test, dummy_train, dummy_test = train_test_split(rest_col,dummy , test_size=0.205, shuffle=False)
    #return rest_test
    rest_test.index = range(27) 
  
    
    
    X_train_vals = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))
    X_test_vals = X_test.values.reshape((X_test.shape[0], 1, X_test.shape[1]))
    
    def build_model():
        model = Sequential()    
        model.add(LSTM(50, input_shape = (X_train_vals.shape[1], X_train_vals.shape[2])))
        model.add(Dense(1))
        model.add(Dropout(0.2))
        model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mape'])
        return model

    model = build_model()

    history = model.fit(X_train_vals, y_train.values,  epochs=200, 
    batch_size=1,validation_data=(X_test_vals, y_test.values),verbose=2,
    shuffle=False)
    
    y_pred = model.predict(X_test_vals)
    #return y_pred
    X_test_vals = X_test_vals.reshape(X_test_vals.shape[0],X_test_vals.shape[2]*X_test_vals.shape[1])
    
    inv_new = np.concatenate( (y_pred, X_test_vals) , axis =1) 
    return inv_new
    inv_new = inv_new[:,0:2]
    inv_new = scaler.inverse_transform(inv_new)
    output = inv_new
    output = np.delete(output,0,axis=0)
    #rest_test.drop(rest_test.index[0])  remove 1st row
 
      

    #output = inv_new
    output_df = pd.DataFrame({'predicted':output[:,0],'actual':output[:,1]})
    output_df = pd.concat([output_df, rest_test], axis=1)
    output_df['mape'] = abs(output_df['predicted']-output_df['actual'])/output_df['actual']


<a id="8"></a>

## Performance (Basic to advanced)


|Metrices | Seasonal ARIMAX | LSTM |
| --- | --- | --- |
| Accuracy | 90% | 86% | 
| Caveat | Time consuming |Faster than ARIMA but requires high GPU|
| Status | Accepted as baseline model | Accepted|

<div class="alert alert-block alert-info">
<b>Bonus Tip:</b> For LSTM to fucntion, you would need to manually decide the timesteps for which you want LSTM
    to remember the informaion, if the data is for 52 weeks, you might want to just look at previous 3 weeks to 
    predict the future data and create the lag variable accrodingly. The whole point of LSTM is to create a lag 
    variable smartly,domain knowledge plays a very important role here </div>